# dataset analysis

In [2]:
# use manga109api parser
import manga109api
manga109_root_dir = "."
manga109_parser = manga109api.Parser(root_dir=manga109_root_dir)

In [4]:
# create vocab / char list 
import re
from natsort import natsorted

for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()
    
    count_polygon = 0
    vocab_list = []
    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        for page_index in range(page_number):
            try:
                rois = annotation["page"][page_index]["onomatopoeia"]
                if isinstance(rois, dict):
                    rois = [rois]  # for one instance case.
            except:
                continue

            for roi in rois:
                label = roi["#text"]
                vocab_list.append(label)

    vocab_set = set(vocab_list)            
    sorted_vocab_set = natsorted(vocab_set)

    with open(f"./Onomatopoeia_{data_split}_vocab_set.txt", "w", encoding="utf-8") as vocab_file:
        join_sorted_char_set = "\n".join(sorted_vocab_set)
        vocab_file.write(join_sorted_char_set)

    # len(vocab_list), len(vocab_set), sorted_vocab_set
    print(data_split, len(vocab_list), len(vocab_set))
    
    # char list
    char_list = []
    for vocab in vocab_list:
        for char in vocab:
            char_list.append(char)

    char_set = set(char_list)
    sorted_char_set = natsorted(char_set)
    print(data_split, len(char_list), len(char_set), sorted_char_set)

    with open(f"./Onomatopoeia_{data_split}_char_set.txt", "w", encoding="utf-8") as char_file:
        join_sorted_char_set = "".join(sorted_char_set)
        char_file.write(join_sorted_char_set)

total 61465 13272
total 174023 182 ['!', '.', '?', '~', '★', '☆', '♡', '♥', '♩', '♪', '♫', '♬', '、', '。', 'ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'ゔ', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'ゎ', 'わ', 'を', 'ん', '゛', '゜', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ヴ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ヮ', 'ワ', 'ン', 'ヶ', '・', 'ー']
train 50064 11635
train 140631 182 ['!', '.', '?', '~'

In [3]:
# Count polygon type. 
# There is only 1 triangle polygon (= polygon consists of 3 points).
for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()

    count_pages = 0

    count_polygon = 0 
    polygon_total_point = 0

    count_over_4points = 0
    count_over_4points_total_point = 0
    count_rectangle = 0
    count_quad = 0

    max_point = 0
    max_polygon_per_page = 0 

    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        for page_index in range(page_number):
            count_pages += 1
            polygon_per_page = 0

            try:
                rois = annotation["page"][page_index]["onomatopoeia"]
                if isinstance(rois, dict):
                    rois = [rois]  # for one instance case.
            except:
                continue

            for roi in rois:
                count_polygon += 1
                polygon_per_page += 1

                x_list = [int(roi[attr]) for attr in roi if "@x" in attr]
                y_list = [int(roi[attr]) for attr in roi if "@y" in attr]

                polygon_total_point += len(x_list) # all polygon

                if len(x_list) != len(y_list):
                    assert Error

                if len(x_list) < 4: # only 1 triangle polygon
                    print(data_split, roi["@id"], len(x_list), f"{manga_name}_{page_index}")

                elif (x_list[0] == x_list[3] and x_list[1] == x_list[2]) and \
                    (y_list[0] == y_list[1] and y_list[2] == y_list[3]):
                    count_rectangle += 1

                elif len(x_list) == 4:
                    count_quad += 1
                else:
                    if len(x_list) > max_point:
                        max_point = len(x_list)
                        # print("max point update", data_split, roi["@id"], len(x_list), f"{manga_name}_{page_index}")

                    count_over_4points += 1
                    count_over_4points_total_point += len(x_list)

            if polygon_per_page > max_polygon_per_page:
                max_polygon_per_page = polygon_per_page
                # print(max_polygon_per_page, manga_name, page_index, roi["@id"])

    average_point = polygon_total_point / count_polygon
    average_over_4points = count_over_4points_total_point / count_over_4points
    over_4points_percentage = count_over_4points/count_polygon * 100
    print(data_split, count_polygon, count_pages, count_polygon/count_pages, average_point, average_over_4points, max_point, max_polygon_per_page)
    print(data_split, count_over_4points, count_rectangle, count_quad, over_4points_percentage)

total 10004784 3 HealingPlanet_24
total 61465 10602 5.797491039426523 6.269096233628894 7.704409030544489 66 82
total 37650 14325 9489 61.25437240706093
train 10004784 3 HealingPlanet_24
train 50064 8763 5.713111947963027 6.3240252476829655 7.725377817622951 66 74
train 31232 11080 7751 62.384148290188556
val 4636 890 5.2089887640449435 6.390854184641933 7.822068965517241 56 38
val 2900 1037 699 62.553925798101815
test 6765 949 7.128556375131717 5.779157427937916 7.421262080727686 43 82
test 3518 2208 1039 52.00295639320029


In [5]:
# count link annotations
for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()

    count_link = 0
    count_link_length2 = 0
    count_link_length3 = 0
    count_link_length4 = 0
    count_link_length5 = 0
    total_characters = 0
    total_page = 0
    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        total_page += page_number
        for page_index in range(page_number):
            id_to_text = {}
            count_onomatopoeia_per_page = 0
            for annotation_type in ["onomatopoeia", "onomatopoeia_link1", "onomatopoeia_link2"]:
                try:
                    rois = annotation["page"][page_index][annotation_type]
                    if isinstance(rois, dict):
                        rois = [rois]  # for one instance case.
                        
                except:
                    continue
                
                for roi in rois:
                    if annotation_type == "onomatopoeia":
                        id_to_text[roi["@id"]] = roi["#text"]
                        count_onomatopoeia_per_page += 1

                    elif "onomatopoeia_link" in annotation_type:
                        linked_text = ""
                        link_id_list = [roi[attr] for attr in roi if "@link" in attr]
                        for id in link_id_list:
                            linked_text += id_to_text[id]

                        total_characters += len(linked_text)

                        length_link_id = len(link_id_list)
                        count_link +=1

                        if length_link_id == 2:
                            # print(length_link_id, manga_name, page_index, roi["@id"])
                            count_link_length2 +=1
                        
                        if length_link_id == 3:
                            # print(length_link_id, manga_name, page_index, roi["@id"])
                            count_link_length3 +=1
                            
                        if length_link_id == 4:
                            # print(length_link_id, manga_name, page_index, roi["@id"])
                            count_link_length4 +=1
                            
                        if length_link_id == 5:
                            # print(length_link_id, manga_name, page_index, roi["@id"])
                            count_link_length5 +=1
                            
    
    print(data_split, total_page, count_link, count_link/total_page, count_link_length2, count_link_length3, count_link_length4, count_link_length5)

total 10602 2261 0.2132616487455197 2117 132 11 1
train 8763 1923 0.21944539541252994 1799 113 10 1
val 890 161 0.18089887640449437 152 9 0 0
test 949 177 0.18651211801896733 166 10 1 0


In [6]:
# count reading link left-to-right
for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()
    
    count_link = 0
    count_left_to_right_link = 0
    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        for page_index in range(page_number):
            id_to_text = {}
            id_to_center_x = {}
            
            for annotation_type in ["onomatopoeia", "onomatopoeia_link1", "onomatopoeia_link2"]:
                try:
                    rois = annotation['page'][page_index][annotation_type]
                    if isinstance(rois, dict):
                        rois = [rois]  # for one instance case.
                        
                except:
                    continue

                for roi in rois:
                    if annotation_type == "onomatopoeia":
                        id_to_text[roi["@id"]] = roi["#text"]
                        x_list = [int(roi[attr]) for attr in roi if "@x" in attr]
                        
                        center_x = int(sum(x_list) / len(x_list))
                        id_to_center_x[roi["@id"]] = center_x
                        
                    elif "onomatopoeia_link" in annotation_type:
                        count_link +=1
                        
                        link_id_list = [roi[attr] for attr in roi if "@link" in attr]
                        if id_to_center_x[link_id_list[0]] < id_to_center_x[link_id_list[1]]:
                            count_left_to_right_link += 1
            
    print(data_split, count_left_to_right_link, count_left_to_right_link/count_link*100)

total 1787 79.0358248562583
train 1518 78.93915756630265
val 131 81.36645962732919
test 138 77.96610169491525


In [7]:
# count characters in each word
import pprint
pp = pprint.PrettyPrinter(indent=4)

for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()
    
    count_char_length = {}
    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        for page_index in range(page_number):
            for annotation_type in ["onomatopoeia"]:
                try:
                    rois = annotation["page"][page_index][annotation_type]
                    if isinstance(rois, dict):
                        rois = [rois]  # for one instance case.
                except:
                    continue

                for roi in rois:
                    text = roi["#text"]
                    try:
                        count_char_length[len(text)] += 1
                    except:
                        count_char_length[len(text)] = 1
    
    print(data_split)
    pp.pprint(count_char_length)

total
{   1: 5845,
    2: 26064,
    3: 15909,
    4: 6713,
    5: 3384,
    6: 2018,
    7: 747,
    8: 410,
    9: 183,
    10: 75,
    11: 38,
    12: 30,
    13: 14,
    14: 7,
    15: 12,
    16: 3,
    17: 4,
    18: 3,
    19: 1,
    20: 2,
    21: 2,
    28: 1}
train
{   1: 5297,
    2: 20941,
    3: 12844,
    4: 5400,
    5: 2732,
    6: 1631,
    7: 605,
    8: 316,
    9: 152,
    10: 54,
    11: 34,
    12: 23,
    13: 11,
    14: 2,
    15: 10,
    16: 1,
    17: 3,
    18: 2,
    19: 1,
    20: 2,
    21: 2,
    28: 1}
val
{   1: 237,
    2: 2060,
    3: 1264,
    4: 488,
    5: 283,
    6: 154,
    7: 67,
    8: 52,
    9: 13,
    10: 7,
    11: 2,
    12: 3,
    13: 1,
    14: 3,
    15: 1,
    16: 1}
test
{   1: 311,
    2: 3063,
    3: 1801,
    4: 825,
    5: 369,
    6: 233,
    7: 75,
    8: 42,
    9: 18,
    10: 14,
    11: 2,
    12: 4,
    13: 2,
    14: 2,
    15: 1,
    16: 1,
    17: 1,
    18: 1}


In [8]:
# Vertical count
for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()
    
    count_polygon = 0
    count_vertical = 0 # the number of images whose height > width

    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")

        page_number = len(annotation['page'])
        for page_index in range(page_number):
            try:
                rois = annotation["page"][page_index]["onomatopoeia"]
                if isinstance(rois, dict):
                    rois = [rois]  # for one instance case.
            except:
                continue

            for roi in rois:
                count_polygon += 1
                label = roi["#text"]
                x_list = [int(roi[attr]) for attr in roi if "@x" in attr]
                y_list = [int(roi[attr]) for attr in roi if "@y" in attr]

                x_min = min(x_list)
                y_min = min(y_list)
                x_max = max(x_list)
                y_max = max(y_list)

                width = x_max - x_min
                height = y_max - y_min

                if height > width:
                    count_vertical += 1

    print(data_split, count_polygon, count_vertical, count_vertical/count_polygon)

total 61465 44584 0.7253558935979826
train 50064 36383 0.7267297858740812
val 4636 3355 0.7236842105263158
test 6765 4846 0.7163340724316334


In [9]:
# vocab count
import pprint
pp = pprint.PrettyPrinter(indent=4)

for data_split in ["total", "train", "val", "test"]:
    with open(f"{manga109_root_dir}/books_{data_split}.txt", "r") as book_list:
        manga_list = book_list.readlines()

    count_polygon = 0
    vocab = {}
    for manga_name in manga_list:
        manga_name = manga_name.strip()
        annotation = manga109_parser.get_annotation(book=manga_name, annotation_type="annotations")
        
        page_number = len(annotation['page'])
        for page_index in range(page_number):
            try:
                rois = annotation["page"][page_index]["onomatopoeia"]
                if isinstance(rois, dict):
                    rois = [rois]  # for one instance case.
            except:
                continue

            for roi in rois:
                count_polygon += 1
                label = roi["#text"]
                if label in vocab:
                    vocab[label] += 1
                else:
                    vocab[label] = 1


    print(data_split, count_polygon)
    # print(vocab)
    vocab_sorted = []
    for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)[:20]:
        vocab_sorted.append([k, v])

    pp.pprint(vocab_sorted)

total 61465
[   ['オ', 891],
    ['ザワ', 749],
    ['パチ', 687],
    ['ワー', 677],
    ['ドキ', 365],
    ['ン', 347],
    ['はっ', 346],
    ['?', 333],
    ['ゴ', 309],
    ['はあ', 308],
    ['バッ', 305],
    ['ザッ', 296],
    ['ハア', 280],
    ['ド', 273],
    ['ッ', 269],
    ['ゴロ', 256],
    ['ドン', 247],
    ['ガッ', 233],
    ['わい', 233],
    ['ハァ', 222]]
train 50064
[   ['オ', 887],
    ['ワー', 586],
    ['ザワ', 437],
    ['ン', 307],
    ['パチ', 304],
    ['ゴ', 296],
    ['?', 289],
    ['はっ', 289],
    ['ドキ', 285],
    ['バッ', 261],
    ['ド', 247],
    ['ザッ', 246],
    ['ッ', 238],
    ['ゴロ', 224],
    ['ハア', 224],
    ['ガッ', 210],
    ['ドン', 204],
    ['はあ', 202],
    ['ア', 199],
    ['!', 187]]
val 4636
[   ['わい', 75],
    ['ザワ', 71],
    ['パチ', 58],
    ['ドキ', 52],
    ['ハア', 37],
    ['ざわ', 37],
    ['ガヤ', 35],
    ['ザッ', 35],
    ['はっ', 31],
    ['ガラッ', 30],
    ['ワイ', 29],
    ['ドキン', 27],
    ['ガタ', 27],
    ['プル', 27],
    ['どき', 26],
    ['バタ', 26],
    ['パタ', 24],
    ['!', 23],
    ['バッ', 2